# My first project - using vaults

When creating a project, we start defining the infrastructure layout via two files:

  - ansible.cfg
  - inventory


We're going to describe the following setup:

  - direct access to client machines (eg. no bastion)
  - one `inventory` file
  - no host_key_check
  - no retry files
  - client machine username/password is `root:root`
  - no public key is installed by default



In [ ]:
cd /notebooks/exercise-01/

## Preparation

The first steps include creating:

 - all secret files (ssh identity, vault pin file)
 - ansible.cfg (the deployment descriptor) 
 - and the inventory.

In [ ]:
# At first create a proper ssh key for the project
! rm id_ansible.pub -r
! test -f id_ansible || ssh-keygen -q -t ecdsa -f id_ansible  -N ''

In [ ]:
# Now a `secret` password ;) We'll reference this in ansible.cfg
!echo secret > .pin    

In [ ]:
# then prepare a deployment descriptor referencing the .pin file
!cat ansible.cfg

### Exercise

 - use ansible docs or the web to comment properly all the [ansible.cfg](/edit/notebooks/exercise-01/ansible.cfg) params

In [ ]:
# And in the end, the inventory. 
!cat inventory

## Gotta ping 'em all

Everything set up now. 

Ping all hosts now, eventually adjusting ip ranges in then [inventory](/edit/notebooks/exercise-01/inventory).

In [ ]:
# Let's get an error: root_password is UNDEFINED. 
!ansible -m ping all


## Vaults

We now ensure that everything is encrypted and we're able to connect to some hosts

In [ ]:
# A vault is just a yaml file containing a dictionary of secrets.
#  We can put here as many information as we want, but for now
#  just put the `root_password`.

!echo "root_password: root" > vault.yml

In [ ]:
# We need to encrypt it. 
!ansible-vault encrypt vault.yml

### Exercise

  - which secret is used to encrypt vaultfile.yml ?
  - where is it specified?
  - what happens if you try to re-encrypt the vault.yml

In [ ]:
# And show the anatomy of the vault.
!cat vault.yml

In [ ]:
# Can we decrypt it?
!ansible-vault view vault.yml

In [ ]:
!ansible -m ping all -e@vault.yml

## Exercise

 - How can you pass the vault password file from the command line?
 - Run ansible in verbose mode and check how ansible pass the password to ssh: does it use some helper program?

In [ ]:
# Write answers here

## SSH Authentication

We want to switch from password to ssh authentication. Create a playbook to install ssh keys: it reads the password from vault.yml


In [ ]:
!cat copy-key.yml

In [ ]:
!ansible-playbook copy-key.yml

### Exercise

Comment out the ansible_password field  [in `inventory` here](/edit/notebooks/exercise-01/inventory)

  - guess the expected output without running ansible
  
  

In [ ]:
# Running in debug mode we can see all ssh arguments injected via anisble. Discuss the DEBUG output       
!sed -i 's/ansible_password/#ansible_password/' inventory
!ansible -vvv -m ping all

In [ ]:
# Use this cell for the exercise

## Exercise

Run ansible in verbose mode to se all the injected ssh argument. If the output is too verbose, reduce it either with:
    - `--limit ipaddress` to contact only one node
    - host indexing/subscript eg: `all[0]` 

# Discuss vaults for:
- common secrets to be shared on a repo
- private secrets to reside on local PC